In [ ]:
# KOMÓRKA 1: INSTALACJA I URUCHOMIENIE (WERSJA Z POPRAWKĄ CPU/GPU)

print("KROK 1/5: Aktualizowanie i instalowanie niezbędnych bibliotek...")
!pip install -q -U diffusers transformers accelerate bitsandbytes gradio
!apt -y install -qq aria2

print("\nKROK 2/5: Pobieranie modelu LoRA...")
LORA_DIR = "/content/models/loras"
!mkdir -p {LORA_DIR}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux_realism_lora.safetensors -d {LORA_DIR} -o flux_realism_lora.safetensors

import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import gradio as gr
from PIL import Image
import random

print("\nKROK 3/5: Ładowanie głównego modelu FLUX (to może potrwać najdłużej)...")
pipe = DiffusionPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True
)

print("\nKROK 4/5: Konfiguracja dodatkowych komponentów...")
# Konfiguracja samplera
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Ładowanie wag LoRA z nadaniem nazwy adaptera
pipe.load_lora_weights(LORA_DIR, weight_name="flux_realism_lora.safetensors", adapter_name="realism")

# OSTATECZNA POPRAWKA: Jawne przeniesienie całego potoku na GPU, aby uniknąć konfliktu CPU/CUDA
print("Wymuszanie pracy potoku na urządzeniu GPU (cuda)...")
pipe.to("cuda")

# Definicja funkcji generującej obraz
def generate(positive_prompt, negative_prompt, width, height, seed, steps, guidance, lora_strength):
    if seed == 0:
        seed = random.randint(0, 2**32 - 1)
    print(f"Użyte parametry: Ziarno={seed}, Kroki={steps}, CFG={guidance}, Siła LoRA={lora_strength}")

    pipe.set_adapters(["realism"], adapter_weights=[lora_strength])

    generator = torch.Generator("cuda").manual_seed(seed)

    image = pipe(
        prompt=positive_prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        num_inference_steps=steps,
        guidance_scale=guidance,
        generator=generator
    ).images[0]

    return image

# Budowanie interfejsu Gradio
with gr.Blocks(analytics_enabled=False, theme=gr.themes.Default(primary_hue="blue")) as demo:
    gr.Markdown("# Interfejs do generowania obrazów z modelu FLUX")
    with gr.Row():
        with gr.Column(scale=1):
            positive_prompt = gr.Textbox(lines=5, interactive=True, value="A captivating, photorealistic image with RAW quality, cinematic grain, and eye-catching light. The scene features a 26-year-old hippie woman from 1966, radiating seductive, retro charm. She wears vibrant, psychedelic attire—a boldly patterned blouse with bell sleeves and a matching mini-skirt. Her long hair is styled in a classic '60s updo, accented with a headband, completing her look. Posing confidently in front of an orange american muscle car of 1960s, surrounded by industrial buildings on the seaport, her smile and gaze exude an irresistible allure. This image perfectly captures the essence of 1960s counterculture, blending nostalgia with timeless beauty. The shoot features vibrant colors, hyper-detailed dynamic style, with a glamorous and boudoir flair", label="Prompt")
            negative_prompt = gr.Textbox(lines=3, interactive=True, value="(deformed, distorted, disfigured:1.3), poorly drawn, bad anatomy, wrong anatomy, extra limb, missing limb, floating limbs, (mutated hands and fingers:1.4), disconnected limbs, mutation, mutated, ugly, disgusting, blurry, amputation", label="Negative Prompt")
            
            with gr.Row():
                steps = gr.Slider(minimum=4, maximum=50, value=30, step=1, label="Kroki (Steps)")
                guidance = gr.Slider(minimum=0, maximum=20, value=5.0, step=0.5, label="Skala CFG (Guidance)")
            
            with gr.Row():
                width = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="Szerokość")
                height = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="Wysokość")
            
            with gr.Row():
                seed = gr.Slider(minimum=0, maximum=4294967295, value=425215417, step=1, label="Ziarno (seed, 0=losowe)")
                lora_strength = gr.Slider(minimum=0, maximum=1.5, value=1.0, step=0.1, label="Siła LoRA")
            
            generate_button = gr.Button("Generuj Obraz", variant="primary")

        with gr.Column(scale=1):
            output_image = gr.Image(label="Wygenerowany obraz", interactive=False, height=512)
            
    generate_button.click(fn=generate, inputs=[positive_prompt, negative_prompt, width, height, seed, steps, guidance, lora_strength], outputs=output_image)

print("\nKROK 5/5: Uruchamiam interfejs Gradio...")
demo.queue().launch(inline=False, share=True, debug=True)